# Mission to Mars

### Import Dependencies

In [62]:
import pandas as pd
import pymongo as pmo
from bs4 import BeautifulSoup as bsp
import requests
from splinter import Browser

In [3]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = bsp(response.text, 'html5lib')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <title>
   News - NASA Mars
  </title>
  <link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
  <link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
  <link href="/manifest.json" rel="manifest"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="#000000" name="theme-color"/>
  <meta content="en" http-equiv="content-language"/>
  <meta content="News" property="og:title"/>
  <meta content="mars.nasa.gov" name="creator"/>
  <meta content="mars.nasa.gov" name="author"/>
  <meta content="NASA, Mars, Perseverance, rover, 2020, mission, astrobiology, life, human, exploration" name="keywords"/>
  <meta content="NASA's Mars 2020 Perseverance rover will look for signs of past microbial life, cache rock and soil samples, and prepare for future human exploration." name="description"/>
  <meta content="NASA's M

In [34]:
results = soup.find_all('div', class_="listTextLabel")
results

[<div class="listTextLabel">
 <h2 class="alt01"><a href="https://mars.nasa.gov/news/8654/">How NASA's Perseverance Mars Team Adjusted to Work in the Time of Coronavirus </a></h2>
 <p>Like much of the rest of the world, the Mars rover team is pushing forward with its mission-critical work while putting the health and safety of their colleagues and community first.</p>
 <div class="newsSource">
 <p class="source">JPL/NASA-Caltech | </p>
 <p class="date">April 21, 2020</p>
 </div>
 </div>,
 <div class="listTextLabel">
 <h2 class="alt01"><a href="https://mars.nasa.gov/news/8649/">NASA's Perseverance Mars Rover Gets Balanced</a></h2>
 <p>The mission team performed a crucial weight-balancing test on the rover in preparation for this summer's history-making launch to the Red Planet.</p>
 <div class="newsSource">
 <p class="source">JPL/NASA-Caltech | </p>
 <p class="date">April 20, 2020</p>
 </div>
 </div>,
 <div class="listTextLabel">
 <h2 class="alt01"><a href="https://mars.nasa.gov/news/864

In [39]:
results[0].h2.a.text

"How NASA's Perseverance Mars Team Adjusted to Work in the Time of Coronavirus "

In [41]:
results[0].p

<p>Like much of the rest of the world, the Mars rover team is pushing forward with its mission-critical work while putting the health and safety of their colleagues and community first.</p>

In [52]:
# Loop through returned results
for result in results:
    # Error handling
    try:
        # get the article title
        news_title = result.h2.a.text
        # get the article content
        news_content = result.p.text
        # Identify and return link to listing
        news_link = result.h2.a['href']
        print (f'{("-")*50}\n> Title:\t {news_title}\n>> Link:\t {news_link}\n>>> Content:\t {news_content}')

    except AttributeError as e:
        print(e)

--------------------------------------------------
> Title:	 How NASA's Perseverance Mars Team Adjusted to Work in the Time of Coronavirus 
>> Link:	 https://mars.nasa.gov/news/8654/
>>> Content:	 Like much of the rest of the world, the Mars rover team is pushing forward with its mission-critical work while putting the health and safety of their colleagues and community first.
--------------------------------------------------
> Title:	 NASA's Perseverance Mars Rover Gets Balanced
>> Link:	 https://mars.nasa.gov/news/8649/
>>> Content:	 The mission team performed a crucial weight-balancing test on the rover in preparation for this summer's history-making launch to the Red Planet.
--------------------------------------------------
> Title:	 Mars Helicopter Attached to NASA's Perseverance Rover
>> Link:	 https://mars.nasa.gov/news/8645/
>>> Content:	 The team also fueled the rover's sky crane to get ready for this summer's history-making launch.
------------------------------------------

In [54]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

UnboundLocalError: local variable 'e' referenced before assignment